In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# install pyspark
!pip3 install pyspark==3.2.0

# install graphframes
!pip3 install graphframes

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 68.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6a277d11ef5043bf197e3117a8429c637a7d12b59ef565324e145dc290a9d9ca
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 154 kB 30.4 MB/s 


In [ ]:
!cp -v /content/graphframes-0.8.2-spark3.2-s_2.12.jar $SPARK_HOME/jars/

'/content/graphframes-0.8.2-spark3.2-s_2.12.jar' -> '/content/spark-3.2.0-bin-hadoop3.2/jars/graphframes-0.8.2-spark3.2-s_2.12.jar'


In [ ]:
#import the packages
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import pandas as pd

findspark.init()

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import psycopg2
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='cpdbstudent', password='DataSci4AI', host='codd01.research.northwestern.edu', port= '5432'
)

In [ ]:
cursor = conn.cursor()

In [ ]:
edges_query = '''
DROP TABLE IF EXISTS ids_distinct2;
create temp table ids_distinct2 as(
select distinct a.id
   	from data_officer a
join (
Select POLICE_UNIT, COUNT_ALLEGATIONS, total_officers, TRUNC((COUNT_ALLEGATIONS :: numeric / total_officers),2) as allegation_ratio_per_unit  from
(select b.last_unit_id AS POLICE_UNIT, count(a.officer_id) AS COUNT_ALLEGATIONS from data_officerallegation a
join data_officer b
on a.officer_id = b.id
group by b.last_unit_id
order by count(a.officer_id) desc)A

Inner join

(Select count(id) as total_officers, last_unit_id from data_officer group by last_unit_id order by  last_unit_id)B

on A.POLICE_UNIT = B.last_unit_id order by allegation_ratio_per_unit desc
limit 10) pu
on a.last_unit_id = pu.POLICE_UNIT);

select src, dst, relationship from(
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC)popln_sample2
JOIN ids_distinct2 ON ids_distinct2.id = popln_sample2.src;

'''

In [ ]:
nodes_query = '''
DROP TABLE IF EXISTS ids_distinct2;
create temp table ids_distinct2 as(
select distinct a.id
   	from data_officer a
join (
Select POLICE_UNIT, COUNT_ALLEGATIONS, total_officers, TRUNC((COUNT_ALLEGATIONS :: numeric / total_officers),2) as allegation_ratio_per_unit  from
(select b.last_unit_id AS POLICE_UNIT, count(a.officer_id) AS COUNT_ALLEGATIONS from data_officerallegation a
join data_officer b
on a.officer_id = b.id
group by b.last_unit_id
order by count(a.officer_id) desc)A

Inner join

(Select count(id) as total_officers, last_unit_id from data_officer group by last_unit_id order by  last_unit_id)B

on A.POLICE_UNIT = B.last_unit_id order by allegation_ratio_per_unit desc
limit 10) pu
on a.last_unit_id = pu.POLICE_UNIT);

DROP TABLE IF EXISTS edges_q2;
create temp table edges_q2 as(
select src, dst, relationship from(
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC)popln_sample2
JOIN ids_distinct2 ON ids_distinct2.id = popln_sample2.src);

SELECT  id, last_unit_id, first_name || ' ' || last_name officer_name, allegation_count FROM data_officer
where id in (select distinct src from edges_q2);

'''

In [ ]:
cursor.execute(edges_query)
edges = cursor.fetchall()
print("shape is: " + str(len(edges))) # 17465

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames

shape is: 2776


In [ ]:
cursor.execute(nodes_query)
nodes = cursor.fetchall()
print("shape is: " + str(len(nodes))) # 17465

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames

shape is: 125


In [ ]:
df_nodes

,id,last_unit_id,officer_name,allegation_count
0,28298,266,Brian Tedeschi,11
1,8769,175,Jose Flores,17
2,11427,174,Derrick Harland,7
3,1172,174,Damen Balesteri,58
4,867,174,Reginald Arrington,23
...,...,...,...,...
120,5571,173,Bryan Cox,26
121,32310,174,Bradley Ruzak,26
122,28203,266,Johnny Tate,12
123,6221,174,Herbert Darey,25


In [ ]:
edges_ = spark.createDataFrame(df_edges)

In [ ]:
nodes = spark.createDataFrame(df_nodes)

In [ ]:
cpdb2 = GraphFrame(nodes, edges_)

In [ ]:
tc_cpdb = cpdb2.triangleCount()
tc_cpdb.orderBy('count', ascending=False).show()
tc_cpdb.select("id", "count").show()

+-----+-----+------------+------------------+----------------+
|count|   id|last_unit_id|      officer_name|allegation_count|
+-----+-----+------------+------------------+----------------+
|    7|18176|         174|       John Mc Gee|              34|
|    7| 3395|         173|        Adam Burns|              16|
|    7| 1172|         174|   Damen Balesteri|              58|
|    6|30537|         174|   Tracy Whitehead|              19|
|    6|10558|         266|    Anthony Granat|              21|
|    6|27385|         174|     Bill Starling|              29|
|    5|11641|         174|       Craig Hatch|               7|
|    5|12220|         173|     Courtney Hill|              72|
|    4|27439|         173| Robert Stegmiller|              62|
|    4| 2671|         173|      Sean Brandon|              40|
|    4|10663|         174|       Andre Green|              36|
|    4|23600|         173|        Brian Reed|              23|
|    3| 2588|         174|    Jason Bradford|          

In [ ]:
pr_cpdb = cpdb2.pageRank(resetProbability=0.15, tol=0.01)
#look at the pagerank score for every vertex
pr_cpdb.vertices.orderBy('pagerank', ascending=False).show()
pr_cpdb.edges.show()

+-----+------------+-----------------+----------------+------------------+
|   id|last_unit_id|     officer_name|allegation_count|          pagerank|
+-----+------------+-----------------+----------------+------------------+
|31996|         175|     German Gomez|              12| 1.305863820393211|
|25191|         173|    David Salgado|              31|1.1865471624739725|
|27935|         165|    Matthew Swain|              61|1.1847111268651542|
|32406|         173|    Reginald Ward|              17|1.1645157563285098|
|24521|         175|     Carlos Rojas|              71|1.1580852338143208|
|27439|         173|Robert Stegmiller|              62|1.1577471014734335|
|30320|         180| Robert Weisskopf|              23|1.1524574235387521|
|29445|         173|        Luis Vega|              53|1.1440847317012035|
|30537|         174|  Tracy Whitehead|              19|1.1299951650932252|
|23600|         173|       Brian Reed|              23| 1.121360147183769|
|30387|         173|  Mar